# 80. ID番号への変換

# 81. RNNによる予測

# 82. 確率的勾配降下法による学習

# 83. ミニバッチ化・GPU上での学習

# 84. 単語ベクトルの導入

# 85. 双方向RNN・多層化

# 86. 畳み込みニューラルネットワーク (CNN)

# 87. 確率的勾配降下法によるCNNの学習

# 88. パラメータチューニング

# 89. 事前学習済み言語モデルからの転移学習